### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "models/Qwen3-14B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = False,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = True, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

In [ ]:
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 32,  # Best to choose alpha = rank or rank*2
#     lora_dropout = 0, # Supports any, but = 0 is optimized
#     bias = "none",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,   # We support rank stabilized LoRA
#     loftq_config = None,  # And LoftQ
# )

### Data Prep

In [3]:
from datasets import load_dataset

# 1. 讀取您的 CSV 檔案
# 假設您的 CSV 檔案就在當前目錄下，名為 "my_data.csv"
dataset = load_dataset("csv", data_files="0.6_acc_result.csv", split="train")

# 2. 修改轉換函數，對應您的 CSV 欄位名稱
def generate_conversation(examples):
    # 修改這裡：把 CSV 裡的 "question" 欄位抓出來
    problems  = examples["question"]
    
    # 修改這裡：把 CSV 裡的 "assistant_reply" 欄位抓出來
    solutions = examples["assistant_reply"]
    
    conversations = []
    # 使用 zip 將問題和回答配對
    for problem, solution in zip(problems, solutions):
        conversations.append([
            {"role" : "user",      "content" : problem},
            {"role" : "assistant", "content" : solution},
        ])
    
    return { "conversations": conversations }

# 3. 執行轉換
# 這會產生一個新的 "conversations" 欄位，裡面裝好格式化的對話
dataset = dataset.map(generate_conversation, batched=True)

# print(dataset[0]["conversations"])

In [ ]:
# 這裡確保使用正確的 Chat Template (例如 Qwen, Llama 等)
from unsloth.chat_templates import get_chat_template

# 確保 tokenizer 知道要用哪種格式 (這行通常在載入模型時會做，但保險起見可以加)
tokenizer = get_chat_template(tokenizer, chat_template = "qwen-2.5") 

def formatting_prompts_func(examples):
    conversations = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in conversations]
    return { "text" : texts, }

# 執行轉換：這會產生一個新的 "text" 欄位
dataset = dataset.map(formatting_prompts_func, batched = True)

# 檢查結果：現在模型真正看到的是這個樣子
print(dataset[0]["text"])

### Train the model

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2ez-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)